In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob as g

In [ ]:
files = g('np_outputs/points*npy')
len(files)

list_lane0 = []
list_lane1 = []
list_lane2 = []
list_lane3 = []
for file in files:
    arr = np.load(file, allow_pickle = True)

    lane1 = np.array(arr[1])
    lane2 = np.array(arr[2])
    lane0 = np.array(arr[0])
    lane3 = np.array(arr[3])
    arr = np.load(file, allow_pickle = True)
    
    list_lane0.append(lane0)
    list_lane1.append(lane1)
    list_lane2.append(lane2)
    list_lane3.append(lane3)

In [ ]:
def moving_avg(new_arr, old_arr, new_weight = 0.05):
    return new_arr * new_weight + old_arr * (1-new_weight)


class MvArray():
    def __init__(self, initial_array, alpha):
        self.array = initial_array
        self.cnt = 0
        self.alpha = alpha
    
    def update(self, new_array):
        c = max(self.alpha, 1 / (self.cnt + 1))
        self.array = moving_avg(new_array, self.array, new_weight= c)
        self.cnt += 1
        return self.array

class Line():
    def __init__(self, lines_alpha = 0.05, limits_alpha = 0.05, n_degrees = 3):
        self.n_degrees = n_degrees
        self.cnt = 0
        self.limits_alpha = limits_alpha
        self.lines_alpha = lines_alpha
        
        
    def update(self, new_line):
        if self.cnt == 0 :
            
            if len(new_line):
            
                p = np.polyfit(new_line[:,0], new_line[:,1], self.n_degrees)
                (x_min, _), (x_max, _) = line1.min(0), line1.max(0)

                self.limits = MvArray(np.array([x_min, x_max]), self.limits_alpha)
                x_min, x_max = self.limits.array

                self.axis = np.linspace(x_min, x_max, 10)
                npred_line = np.polyval(p, self.axis) #predicted line
                pred_line = npred_line

                self.predicted_line = MvArray(pred_line, self.lines_alpha)

                self.cnt += 1
        
        else:
            if len(new_line):
                p = np.polyfit(new_line[:,0], new_line[:,1], self.n_degrees)
                (nx_min, _), (nx_max, _) = new_line.min(0), new_line.max(0)

                self.limits.update(np.array([nx_min, nx_max]))
                x_min, x_max = self.limits.array

                self.axis = np.linspace(x_min, x_max, 10)
                npred_line = np.polyval(p, self.axis)

                self.predicted_line.update(npred_line)
                self.cnt += 1
            
    def get_line(self):
        try:
            return np.array([self.axis, self.predicted_line.array]).T
        except:
            return np.array(np.zeros((10,2)))

In [ ]:
%matplotlib qt

# create an instance for each line
myline0 = Line(lines_alpha = 0.05, limits_alpha = 0.05, n_degrees = 3)
myline1=  Line(lines_alpha = 0.05, limits_alpha = 0.05, n_degrees = 3)
myline2=  Line(lines_alpha = 0.05, limits_alpha = 0.05, n_degrees = 3)
myline3 = Line(lines_alpha = 0.05, limits_alpha = 0.05, n_degrees = 3)

for idx , (line0, line1,line2, line3) in enumerate(zip(list_lane0, list_lane1, list_lane2, list_lane3)): 
    
    myline1.update(line1) # for each frame, update the line point values
    myline2.update(line2) # for each frame, update the line point values
    myline3.update(line3) # for each frame, update the line point values
    myline0.update(line0) # for each frame, update the line point values
    
    pline1 = myline1.get_line() #this code gets the line points from the Line object
    pline2 = myline2.get_line()
    pline0 = myline0.get_line()
    pline3 = myline3.get_line()
    
    
    
    # Below is only for plotting using matplotlib #
    #            You do not need these            #
    ###############################################
    if len(line3) == 0:
        line3 = np.array(np.zeros((10,2)))
    if len(line0) == 0:
        line0 = np.array(np.zeros((10,2)))
    
    plt.gca().invert_yaxis()
    plt.xlim(200,1200) 
    plt.ylim(800, 200)
    
    plt.plot(line1[:,0], line1[:,1], label = "actual")
    plt.plot(pline1[:,0], pline1[:,1], label = "mv")
    
    plt.plot(line2[:,0], line2[:,1], label = "actual")
    plt.plot(pline2[:,0], pline2[:,1], label = "mv")
    
    plt.plot(line3[:,0], line3[:,1], label = "actual")
    plt.plot(pline3[:,0], pline3[:,1], label = "mv")
    
    plt.plot(line0[:,0], line0[:,1], label = "actual")
    plt.plot(pline0[:,0], pline0[:,1], label = "mv")
    
    
    plt.pause(0.1) #is necessary for the plot to update for some reason
    
    if idx>2:
        plt.clf()